In [117]:
import numpy as np
import torch.nn.functional as F
import torch

In [118]:
c = 8
n = 4
k = 3
np.random.seed(777)
w = np.random.randint(0, 16, (c, c, k, k)) - 8 # oc, ic, kh, kw
x = np.random.randint(0, 16, (c, n, n)) # ic, nh, nw
x = np.pad(x, ((0,0), (1,1), (1,1)))

w = torch.tensor(w)
x = torch.tensor(x)

psum = F.conv2d(x, w)

# to test that it fails when expected
# w[0,0,0,0] -= 1

with open('x.txt', 'w+') as f:
    for val in x.numpy().transpose(1,2,0).flatten():
        f.write(f"{val:04b}\n")

with open('w.txt', 'w+') as f:
    for val in w.numpy().transpose(2,3,0,1).flatten():
        f.write(f"{val&0xF:04b}\n")

with open('psum.txt', 'w+') as f:
    for val in psum.reshape(c,n*n).numpy().T.flatten():
        f.write(f"{val&0xFFFF:016b}\n")

In [119]:
with open('x_os.txt', 'w+') as f:
    for ic in range(c):
        for ky in range(k):
            for kx in range(k):
                _ = 0
                vals = []
                for ny in range(n):
                    for nx in range(n):
                        vals.append(x[ic,ny+ky,nx+kx])
                        _ += 1
                        if _ == 8: break
                    if _ == 8: break
                for val in vals[::-1]:
                    f.write(f"{val:04b}")
                f.write('\n')

with open('w_os.txt', 'w+') as f:
    for ic in range(c):
        for ky in range(k):
            for kx in range(k):
                for oc in range(min(8, c)):
                    f.write(f"{w[7-oc,ic,ky,kx]&0xF:04b}")
                f.write('\n')

In [120]:
psum_debug = torch.tensor(np.zeros((k*k, c, n, n))) # kij, oc, ny, nx
for ky in range(k):
    for kx in range(k):
        psum_debug[ky*k+kx] = (F.conv2d(x[:,ky:ky+n,kx:kx+n], w[:,:,ky:ky+1,kx:kx+1]))


In [121]:
def hp(arr, digits=4, rev=True):
    try: arr = arr.numpy()
    except: pass
    assert len(arr.shape) <= 2
    if len(arr.shape) == 1: arr = [arr]
    for line in arr:
        for item in line[::-1 if rev else 1]:
            s = hex(int(item) & ((1<<(4*digits))-1))[2:].zfill(digits)
            print(s, end=' ')
        print()

In [122]:
hp(psum.reshape(8,16).T.numpy()[:,::])

ff27 fea8 ff33 fe4e fe8b fed0 ff70 fff6 
ff58 fef6 0089 fe38 ff9c fe33 fe55 feaa 
ffe6 fd84 ffb4 fe1f fef4 fe45 ffa7 ffb1 
febc ff3c ff54 ff19 ffac ff2d ff24 ff99 
fdfd fe8a fe88 fddd fdd3 fe6d ffa9 ffdb 
0000 fd26 ff74 fded ff33 fd2e ff22 ff37 
fec7 fdec 0017 fcb7 feb4 fe19 ff66 fe9a 
fe54 fe7d ff2e fdcf 000b ff5f ff03 0064 
fe8d fee4 000a fe77 fd3e fe75 0012 fff1 
fed5 fea1 003f fc4e fffd feb6 fd7e ff26 
fdd6 ff31 fef3 fcf1 fded ffd2 fec9 fed6 
fd91 fed7 ff13 fd5e 0060 fea4 fee4 002f 
ffca fe71 ff77 feb8 ffaf ff2b 0023 febf 
fee1 fe19 ff4f fe64 008d fe82 fff4 fd89 
ff54 fd85 ff40 fe11 0091 ff15 fe8f fdb4 
ff03 fe04 fea1 fe8c fffa fea6 ff7d ff41 


In [123]:
s,rs = 0,0
oc = 0
nx,ny = 3,1
for ic in range(c):
    hp(w[oc][ic].flatten(), 1, rev=False)
    hp(x[ic][ny:ny+3,nx:nx+3], 1, False)
    s = (w[oc][ic].flatten()*x[ic][ny:ny+3,nx:nx+3].flatten()).sum()
    hp(w[oc][ic].flatten()*x[ic][ny:ny+3,nx:nx+3].flatten(),4,0)
    rs += s
    hp(np.array([s,rs]))
    print()


f 7 3 e f 9 f 5 7 
8 2 0 
f 1 0 
3 9 0 
fff8 000e 0000 ffe2 ffff 0000 fffd 002d 0000 
0011 0011 

c f 1 6 2 0 f a 5 
d 1 0 
a b 0 
d e 0 
ffcc ffff 0000 003c 0016 0000 fff3 ffac 0000 
ffcd ffbc 

6 8 3 9 a c d f 9 
9 6 0 
f 5 0 
7 d 0 
0036 ffd0 0000 ff97 ffe2 0000 ffeb fff3 0000 
ff2a ff5d 

7 3 f 2 a a 6 f 5 
e a 0 
3 f 0 
c 7 0 
0062 001e 0000 0006 ffa6 0000 0048 fff9 0000 
ff97 006d 

c 0 e a 2 4 f 7 0 
1 9 0 
9 7 0 
d a 0 
fffc 0000 0000 ffca 000e 0000 fff3 0046 0000 
ffa4 000d 

8 0 b 2 a 8 b 5 b 
3 5 0 
7 2 0 
1 4 0 
ffe8 0000 0000 000e fff4 0000 fffb 0014 0000 
ff9d fff9 

c 3 8 6 e d 6 4 f 
1 5 0 
8 e 0 
b f 0 
fffc 000f 0000 0030 ffe4 0000 0042 003c 0000 
003a 009d 

d 4 d 0 5 9 c f 2 
e c 0 
d c 0 
5 4 0 
ffd6 0030 0000 0000 003c 0000 ffec fffc 0000 
0064 002a 



In [124]:
hp(psum.reshape(8,-1),rev=0)

fff6 feaa ffb1 ff99 ffdb ff37 fe9a 0064 fff1 ff26 fed6 002f febf fd89 fdb4 ff41 
ff70 fe55 ffa7 ff24 ffa9 ff22 ff66 ff03 0012 fd7e fec9 fee4 0023 fff4 fe8f ff7d 
fed0 fe33 fe45 ff2d fe6d fd2e fe19 ff5f fe75 feb6 ffd2 fea4 ff2b fe82 ff15 fea6 
fe8b ff9c fef4 ffac fdd3 ff33 feb4 000b fd3e fffd fded 0060 ffaf 008d 0091 fffa 
fe4e fe38 fe1f ff19 fddd fded fcb7 fdcf fe77 fc4e fcf1 fd5e feb8 fe64 fe11 fe8c 
ff33 0089 ffb4 ff54 fe88 ff74 0017 ff2e 000a 003f fef3 ff13 ff77 ff4f ff40 fea1 
fea8 fef6 fd84 ff3c fe8a fd26 fdec fe7d fee4 fea1 ff31 fed7 fe71 fe19 fd85 fe04 
ff27 ff58 ffe6 febc fdfd 0000 fec7 fe54 fe8d fed5 fdd6 fd91 ffca fee1 ff54 ff03 
